In [27]:
import argopy
from argopy import DataFetcher
import numpy as np
import os

# Use argopy to download get data in specified region

To use argopy's bgc data set, the usermode needs to be set to 'expert' and the data source needs to be set to 'erddap'

In [2]:
argopy.set_options(src='erddap', mode='expert');

In [35]:
region = {'LatN': 30, 'LatS': 20, 'LonE': -45, 'LonW': -75,
         'minP':0 ,'maxP': 10,
         'startDate': '2020-01','endDate': '2020-02',
         'BGCParams': ['DOXY','BBP700']}

Specify that we want data where listed BGCParams are not NaNs

In [4]:
BGCDataFetcher = DataFetcher(ds='bgc', measured=region['BGCParams'])

## Get data from specified region and output as a pandas dataframe

In [5]:
df = BGCDataFetcher.region([region['LonW'], region['LonE'], region['LatS'], region['LatN'], 
                           region['minP'], region['maxP'], 
                           region['startDate'], region['endDate']]).to_dataframe()

In [6]:
# ds = BGCDataFetcher.region([region['LonW'], region['LonE'], region['LatS'], region['LatN'], 
#                            region['minP'], region['maxP'], 
#                            region['startDate'], region['endDate']]).to_xarray()

## Reformat argopy table to quality control the data

In [7]:
def QualityControlByParam(p, df):
    # p: param name (ex: PRES)
    # df: argopy dataframe
    
    raw = df.loc[:,p].values
    raw_qc = df.loc[:,p+'_QC'].values
    adj = df.loc[:,p+'_ADJUSTED'].values
    adj_qc = df.loc[:,p+'_ADJUSTED_QC'].values
    dmode = df.loc[:,p+'_DATA_MODE'].values

    data = np.zeros(raw.shape[0])*np.NaN
    data_qc = np.zeros(raw.shape[0])*np.NaN

    # Determine if use real-time or delayed-mode data
    data = np.where(dmode=='R',raw,adj)
    data_qc = np.where(dmode=='R',raw_qc,adj_qc)

    # Quality control data
    data = np.where(data_qc<=2, data, np.NaN)

    return data, data_qc

In [8]:
# Reformat argopy data frame
cnames = ['CONFIG_MISSION_NUMBER',
 'CYCLE_NUMBER',
 'DIRECTION',
 'PLATFORM_NUMBER',
 'POSITION_QC',
 'LATITUDE',
 'LONGITUDE',
 'TIME']

df_new = df.loc[:,cnames]

In [30]:
params = ['PRES','TEMP','PSAL','DOXY']
for p in params:
    data, data_qc = QualityControlByParam(p, df)
    df_new[p] = data
    df_new[p+'_QC'] = data_qc

## Save data to csv file specifying data bounds in file name
Data will save in csv_output directory

In [41]:
fname = 'argopy_ouput_LatN_'+str(np.round(region['LatN'],0))+'_LatS_'+str(np.round(region['LatS'],0))+\
'_LonE_'+str(np.round(region['LonE'],0))+'_LonW_'+str(np.round(region['LonW'],0))+\
'_minP_'+str(np.round(region['minP'],0))+'_maxP'+str(np.round(region['maxP'],0))+\
'_SDate_'+region['startDate']+'_EDate_'+region['endDate']+'_PARAMS_'+'_'.join(region['BGCParams'])+'.csv'

In [29]:
if os.path.exists('csv_output/'):
    df_new.to_csv('csv_output/'+fname)
else:
    os.makedirs('csv_output/')
    df_new.to_csv('csv_output/'+fname)